In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# Adding system path
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd().parent.parent))
# sys.path

In [ ]:
# Set to show warnings only once
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Setting up displays
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import math
from plotly import tools
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tabulate import tabulate
color_scheme=["red","blue","green","orange","purple","brown","pink","gray","olive","cyan","darkviolet","goldenrod","darkgreen","chocolate","lawngreen"]

In [ ]:
# Setting up autoreload for libs
%load_ext autoreload
%autoreload 2
%aimport optiml.backend.warehouse_profile

In [ ]:
# Initialize connection to Snowflake and set analysis date
from optiml.connection import SnowflakeConnConfig
connection = SnowflakeConnConfig(accountname='jg84276.us-central1.gcp',warehousename="XSMALL_WH").create_connection()
# Initialize local environment
import os
cache_dir = os.path.expanduser('~/data/knot')

# Analysis setup
sdate = '2022-10-11'
edate = '2022-10-21'
delta = 'hour'
company = 'KNT'
account_type = "enterprise"

# Initialize query library
from optiml.backend.warehouse_profile import WarehouseProfile
wqlib = WarehouseProfile(connection, company, cache_dir)
from optiml.backend.cost_profile import CostProfile, get_previous_dates
cqlib = CostProfile(connection, company, cache_dir, account_type)

print(f"The analysis is carried our for date range {sdate} to {edate}")

### Warehouse load analysis

In [ ]:
wh_configs = wqlib.warehouse_config()
wh_name = wh_configs["name"].unique().tolist()
whs = {}
for n in wh_name:
    whs[n] = wqlib.wh_queued_load_ts(start_date=sdate,end_date=edate,wh_name=n,delta=delta)
    
df = whs[wh_name[0]]
df.head()

In [ ]:
df_credits = cqlib.cost_by_wh_ts(sdate, edate)
df_credits = df_credits[df_credits["warehouse_name"] == wh_name[0]].reset_index(drop=True)
df_credits.sort_values(by=["hourly_start_time"],inplace=True)
df_credits.head()

In [ ]:
trace1 = go.Bar(
    x=df["hourly_start_time"], y=df["avg_queued_load"],
    name='Queued Load',marker=dict(color='rgb(222,0,0)')
)
trace2 = go.Bar(
    x=df["hourly_start_time"], y=df["avg_running_load"],
    name='Running load',marker=dict(color='rgb(0,0,255)')
)
trace3 = go.Scatter(
  x=df_credits['hourly_start_time']
  ,y=df_credits['credits']
#   ,fill='tozeroy'
  ,name='Credits consumed'
)

fig=make_subplots(rows=1,cols=1,shared_yaxes=True)
fig.append_trace(trace1, 1,1)
fig.append_trace(trace2, 1, 1)
fig.update_layout(barmode='stack')
fig.append_trace(trace3, 1,1)
fig.show()

In [ ]:
df_credits.tail()

In [ ]:
df.tail()